<a href="https://colab.research.google.com/github/arcacolab/oneclick/blob/main/%ED%81%B4%EB%A6%AD_%ED%86%B5%ED%95%A9_segsmaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### <font color="orange">***WebUI 설치***</font>

from google.colab import drive
drive.mount('/content/drive')

ui = 'ReForge' # @param ["A1111", "Forge", "ComfyUI", "ReForge", "SwarmUI"]
#SwarmUI는 뭔지 몰라서 모델, 출력물 연동에 해당사항 없음
sd = 'XL' # @#param ["1.5", "XL"] 컨넷 디테일 로라 다운시 버전 확인을 위한 함수인데 필요 없어서 숨겨놓음
civitai___key = '' # @param {type:"string", placeholder:"Civitai API 키를 입력하세요 (선택)"}
hf_read_token = '' # @param {type:"string", placeholder:"Huggingface READ 토큰을 입력하세요 (선택)"}
MODEL_NAME = "NAI-XL_vpred1.0_2dac_final65" # @param ["NoobAI-XL-Vpred-v1.0","MINTSDXL_MintChocoChip_B","NoobAI-XL-v1.1","miaomiaoHarem_v14","uncannyValley_uncannyvallyNoob3dV1","illustriousXLPersonalMerge_v30Noob10based","Illustrious-XL-v0.1","animagineXL40_v40","NoobAI-XL-Vpred-v1.0-perpendicular-cyberfix-v2","hesperidesIllustrious_v10","모델을 선택해주세요","NAI-XL_vpred1.0_2dac_final65"]
LAUNCH = True # @param {"type":"boolean"}

if not civitai___key:
    civitai___key = "qwertyuiopasdfghjklzxcvbnm123456"
    print("토큰이 입력되지 않았습니다.")
    #토큰 입력해도 버그인지 시빗 브라우저에 자동으로 기입이 안되는듯

print(f"civitai___key: {civitai___key}")


!curl -sLo /content/setup.py https://github.com/gutris1/segsmaker/raw/main/script/KC/setup.py
%run /content/setup.py --webui="$ui" --sd="$sd" --civitai_key="$civitai___key" --hf_read_token="$hf_read_token"

if ui == 'A1111':
  PATH = "/content/A1111"
  jadong = True
  args = '--api --xformers --share'

if ui == 'Forge':
  PATH = "/content/Forge"
  jadong = True
  args = '--api --share --disable-xformers --opt-sdp-attention --cuda-stream --pin-shared-memory'

if ui == 'ReForge':
  PATH = "/content/ReForge"
  jadong = True
  args = '--api --share --xformers --cuda-stream --pin-shared-memory'

if ui == 'ComfyUI':
  PATH = "/content/ComfyUI"
  jadong = False
  args = '--dont-print-server --preview-method auto --use-pytorch-cross-attention'

elif ui == 'SwarmUI':
  jadong = False
  args = '--launch_mode none'

if jadong:
  !git clone -q https://github.com/adieyal/sd-dynamic-prompts {PATH}/extensions/sd-dynamic-prompts
  !git clone -q https://github.com/bedovyy/chibi-client {PATH}/extensions/chibi-client


%cd /content
!git clone https://huggingface.co/arcacolab/drive_link

if ui == 'A1111':
  %run /content/drive_link/a1111_drive_link.ipynb
if ui == 'ReForge':
  %run /content/drive_link/reforge_drive_link.ipynb
if ui == 'Forge':
  %run /content/drive_link/forge_drive_link.ipynb
if ui == 'ComfyUI':
  %run /content/drive_link/comfyui_drive_link.ipynb

%cd {PATH}
if jadong == True:
  !aria2c -x 8 -s 8 https://huggingface.co/arcacolab/models/resolve/main/{MODEL_NAME}.safetensors -o /models/Stable-diffusion/{MODEL_NAME}.safetensors
if jadong == False:
  !aria2c -x 8 -s 8 https://huggingface.co/arcacolab/models/resolve/main/{MODEL_NAME}.safetensors -o models/checkpoints/{MODEL_NAME}.safetensors

if LAUNCH:
  %cd -q $WebUI
  %run segsmaker.py {args} #--dont-print-server --preview-method auto --use-pytorch-cross-attention